In [1]:
import pandas as pd
from rapidfuzz import process, fuzz
pd.set_option('display.max_columns', None)

In [2]:
df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")

C:\Users\pepel\AppData\Local\Temp\ipykernel_69628\3719518088.py:1: DtypeWarning: Columns (0,1,3,11,12,19,34,37,39,40,41,42,43,44,45,46,47,49,50,51,52,53,55,56,57,58,59,60,71,73,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")


In [3]:
df_jugadores_info = pd.read_csv("data/jugadores_info.csv")

In [ ]:
df_jugadoresunicos_eventos = df_eventos["player"].unique().tolist()
df_jugadoresunicos_info = df_jugadores_info["playerFullName"].unique().tolist()
df_clubesunicos_eventos = df_eventos["team"].unique().tolist()
df_clubesunicos_info = df_jugadores_info["club"].unique().tolist()

In [ ]:
# Crear listas
lista_1 = df_jugadoresunicos_info
lista_2 = df_jugadoresunicos_eventos

# Copia para gestionar descartes
lista_2_disponibles = lista_2.copy()
nombres_unificados = {}

# Primera pasada (umbral más alto)
for nombre1 in lista_1:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_sort_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 95:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None

# Segunda pasada (umbral medio)
for nombre1 in [k for k, v in nombres_unificados.items() if v is None]:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.partial_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 90:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None

# Tercera pasada (umbral más bajo)
for nombre1 in [k for k, v in nombres_unificados.items() if v is None]:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 85:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None

# Cuarta pasada (umbral más bajo)
for nombre1 in [k for k, v in nombres_unificados.items() if v is None]:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 80:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None

# Quinta pasada (umbral más bajo)
for nombre1 in [k for k, v in nombres_unificados.items() if v is None]:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 75:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None


# Sexta pasada (umbral más bajo)
for nombre1 in [k for k, v in nombres_unificados.items() if v is None]:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 70:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None


# Octava pasada (umbral más bajo)
for nombre1 in [k for k, v in nombres_unificados.items() if v is None]:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 65:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None

# Novena pasada (umbral más bajo)
for nombre1 in [k for k, v in nombres_unificados.items() if v is None]:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 60:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None

# Decima pasada (umbral más bajo)
for nombre1 in [k for k, v in nombres_unificados.items() if v is None]:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 55:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None

# Undecima pasada (umbral más bajo)
for nombre1 in [k for k, v in nombres_unificados.items() if v is None]:
    if not lista_2_disponibles:
        nombres_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 50:
            nombres_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            nombres_unificados[nombre1] = None
    else:
        nombres_unificados[nombre1] = None



# Resultado final
df_nombres = pd.DataFrame(list(nombres_unificados.items()), columns=["Nombre_original", "Nombre_unificado"])


In [42]:

lista_1 = df_clubesunicos_info
lista_2 = df_clubesunicos_eventos

# Copia para gestionar descartes
lista_2_disponibles = lista_2.copy()
clubes_unificados = {}

# Primera pasada (umbral más alto)
for nombre1 in lista_1:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_sort_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 95:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None

# Segunda pasada (umbral medio)
for nombre1 in [k for k, v in clubes_unificados.items() if v is None]:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.partial_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 90:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None

# Tercera pasada (umbral más bajo)
for nombre1 in [k for k, v in clubes_unificados.items() if v is None]:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 85:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None

# Cuarta pasada (umbral más bajo)
for nombre1 in [k for k, v in clubes_unificados.items() if v is None]:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 80:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None

# Quinta pasada (umbral más bajo)
for nombre1 in [k for k, v in clubes_unificados.items() if v is None]:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 75:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None


# Sexta pasada (umbral más bajo)
for nombre1 in [k for k, v in clubes_unificados.items() if v is None]:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 70:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None


# Octava pasada (umbral más bajo)
for nombre1 in [k for k, v in clubes_unificados.items() if v is None]:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 65:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None

# Novena pasada (umbral más bajo)
for nombre1 in [k for k, v in clubes_unificados.items() if v is None]:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 60:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None

# Decima pasada (umbral más bajo)
for nombre1 in [k for k, v in clubes_unificados.items() if v is None]:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 55:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None

# Undecima pasada (umbral más bajo)
for nombre1 in [k for k, v in clubes_unificados.items() if v is None]:
    if not lista_2_disponibles:
        clubes_unificados[nombre1] = None
        continue
    resultado = process.extractOne(nombre1, lista_2_disponibles, scorer=fuzz.token_set_ratio)
    if resultado:
        match, score, _ = resultado
        if score > 50:
            clubes_unificados[nombre1] = match
            lista_2_disponibles.remove(match)
        else:
            clubes_unificados[nombre1] = None
    else:
        clubes_unificados[nombre1] = None



# Resultado final
df_clubes = pd.DataFrame(list(clubes_unificados.items()), columns=["Nombre_original", "Nombre_unificado"])


In [ ]:
# Crear diccionario para mapear los nombres
dict_nombres_info = df_nombres.set_index('Nombre_original')['Nombre_unificado'].to_dict()
# Mapear nombres jugadores
df_jugadores_info["playerFullName"] = df_jugadores_info["playerFullName"].apply(
    lambda x: dict_nombres_info.get(x, x) if dict_nombres_info.get(x) is not None else x)

In [59]:
# Crear diccionario para mapear los equipos
df_equipos_jugadores = df_eventos[["player", "team"]].drop_duplicates()
dict_equipos_jugadores = dict(zip(df_equipos_jugadores["player"], df_equipos_jugadores["team"]))
# Mapear los equipos en la temporada 2015/2016 ya que la info que aparece es actual
df_jugadores_info["club"] = df_jugadores_info["playerFullName"].map(dict_equipos_jugadores)

In [62]:
# Crear diccionarios para mapear los IDs de jugador y equipo
df_equipos = df_eventos[["team", "team_id"]].drop_duplicates()
dict_equipos = dict(zip(df_equipos["team"], df_equipos["team_id"]))
df_jugadores = df_eventos[["player", "player_id"]].drop_duplicates()
dict_jugadores = dict(zip(df_jugadores["player"], df_jugadores["player_id"]))

In [63]:
# Mapear ID Jugadores
df_jugadores_info["playerID"] = df_jugadores_info["playerFullName"].map(dict_jugadores)
# Mapear ID Equipos
df_jugadores_info["clubID"] = df_jugadores_info["club"].map(dict_equipos)

In [65]:
df_jugadores_info.to_csv("data/jugadores_info_normalizado.csv", index=False)